In [1]:
import warnings
warnings.filterwarnings('ignore')

# General packages                                                                                                                                                                                                                                                                                                                                                                                                                        import pandas as pd 
import numpy as np
import pandas as pd
import requests
import time

# regular expressions
import re

# BeautifulSoup
from bs4 import BeautifulSoup

from urllib.parse import urljoin

# Selenium packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#webDriverwait keeps the driver in wait for some time(in sec) given by us, to make url load completely...
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService 
#chromedriver for selinium ...
#whenever we try to open a link using selinium,it opens the url in chrome
#(we can open the url in other browsers also by installing corresponding drivers)
from webdriver_manager.chrome import ChromeDriverManager 

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9'}

# Open tab in full screen (maximized)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless=new') #reads data without opening the browser
chrome_options.add_argument("start-maximized")

In [3]:
# Instantiate Selenium Webdriver
driver = webdriver.Chrome(options = chrome_options)

# Get the main page (Top Restaurants in Mumbai on Zomato)
driver.get("https://www.zomato.com/hyderabad/restaurants")
#driver.get("https://www.zomato.com/hyderabad/somajiguda-restaurants/south-indian?category=1")

SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
#scrolls the page to next scroll height after every 2 sec...
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#time.sleep(100)
# Get page soure
source = driver.page_source

In [4]:
# Get title of  main window
main_window_title = driver.title
print(main_window_title)

Restaurants in Hyderabad - Zomato


In [5]:
#once the url is loaded completely,we use Beautiful soup to extract data
# creating soup
soup = BeautifulSoup(source, "html.parser")
rest_divs = soup.findAll('div', class_='jumbo-tracker')

In [6]:
# creating all the list here according to data
urls = []
rest_name = []
ratings = []
price = []
cuisine = []
no_cuisins = []
delivery_time = []
delivery_tag = '-'

In [7]:
for rest in rest_divs:  

    # name of the restaurant is stored in the h4 tags
    name_tag = rest.find("h4")    
    if name_tag!= None:
        rest_name.append(name_tag.text)
        #print(name_tag.text)
        rating_tag = rest.div.a.next_sibling.div.div.div.div.div.div.div.text
        price_tag = rest.div.a.next_sibling.p.next_sibling.text
        cuisine_tag = rest.div.a.next_sibling.p.text
        try:
            delivery_tag = rest.div.a.next_sibling.div.next_sibling.next_sibling.p.text
        except:
            pass
            
        print(name_tag.text,"====",delivery_tag)
        ratings.append(rating_tag)
        price.append(price_tag)
        cuisine.append(cuisine_tag)
        no_cuisins.append(len(cuisine_tag.split(',')))
        delivery_time.append(delivery_tag)
        
        # links of the restaurants from <a> tags, getting href where links are stored
        link_tag = rest.find("a")
        if link_tag!=None:
            # since we don not get whole link https attached we need to join the link with base
            base = "https://www.zomato.com"  
            try:
                if 'href' in link_tag.attrs:
                    link = link_tag.get('href').replace('/order','')
            except:
                pass             
            url = urljoin(base, link)
            urls.append(url)
            #getting location of the restaurant
            #name_text = ''.join(x for x in name_tag.text if x.isalnum()).lower()
            #link_li = link.split('/')
            #if link_li[1]!='hyderabad':
            #    loc = link_li[1].replace('-','').replace(name_text,'')
            #else:
            #   loc = link_li[2].replace('-','').replace(name_text,'')
                
            #b = re.findall(r'\d+', loc)
            #if len(b)!=0:
            #   location.append(loc.replace(b[0],''))
            #else:
            #    location.append(loc)
            #print(name_text.lower(),'====',loc)    
print(len(rest_name))

Fancy Ice Cream ==== 28 min
Peshawar ==== 37 min
Nazeer Juice Center ==== 27 min
Pista House ==== 35 min
Hotel Nayaab ==== 29 min
New Falak Restaurant ==== 40 min
Subhan Bakery ==== 37 min
Cake D'licious ==== 54 min
Pista House Bakery ==== 33 min
Shah Ghouse Hotel & Restaurant ==== 32 min
Mehfil ==== Currently not accepting orders
Shah Ghouse Special Shawarma ==== 35 min
New King Safina Hotel ==== 30 min
Al Rabea Al Arabi Cafeteria ==== 56 min
MS Bakery & Sweets ==== 32 min
Pizza Hut ==== 57 min
Food Track ==== 47 min
Al Farooj Sonu Kabab ==== 34 min
McDonald's ==== 64 min
Burger King ==== 46 min
Hotel Shadab ==== 31 min
Chotu's The Food People ==== 36 min
Baba Bakery ==== 47 min
Pista House - Premium Lounge ==== 37 min
Al Akbar Fast Food Corner ==== 28 min
Lucky Restaurant ==== 70 min
La Pino'z Pizza ==== 53 min
Scoops Ice Cream ==== 29 min
Masqati Ice Cream ==== 32 min
MS Mandi & Shawarma ==== 40 min
Safeena Bakery & Sweets ==== 39 min
KFC ==== 49 min
Cream Stone Ice Cream ==== 38 mi

Kaati Zone Rolls and Wraps ==== 59 min
Ice Cafe & Lassi Shop ==== 38 min
Al Noor Bakery ==== 40 min
New Babar Bakery ==== 34 min
V Cafe- Meals By PVR ==== 66 min
Kafe Chai Nashta ==== 39 min
Safa Matam Al Mandi ==== 38 min
Shawarma Hot And Cool Point ==== 37 min
Food Bucks ==== 42 min
Burhanpur Khowa Jalebi ==== 41 min
Grill Xpress Restro ==== 48 min
Veg Daawat By Behrouz ==== 49 min
Heaven's Grill ==== 52 min
Daawat Biryani Hub ==== 50 min
Balaji Mahesh Mithai Bhandar ==== 49 min
Vaibhav Tiffins ==== 52 min
Natural Fruits Juices & Salads ==== 48 min
SLAY Coffee ==== 46 min
Stop N Eat Chicken Shawarma ==== 46 min
Zam Zam Arabian Mandi ==== 50 min
Food Court Adda ==== 51 min
Cakes 07 ==== 84 min
Hotel Delhi Dastarkhan ==== 39 min
Peshawar Juice Centre ==== 39 min
Waffle House ==== 68 min
Shiva Chat Bhandar ==== 43 min
Idhika's Cakes & Snacks ==== 39 min
Hotel Delhi Darbar ==== 43 min
Momo Nation ==== 65 min
Zaman's Arabian Food ==== 45 min
New Makkah Tiffin Center ==== 43 min
Voulga Hot

In [8]:
zomato_df = pd.DataFrame({'links': urls, 'names': rest_name, 'ratings': ratings, 'price for one': price, 'cuisine': cuisine, 'no of cisine':no_cuisins, 'delivery time':delivery_time})

In [9]:
zomato_df.to_csv("Hyderabad_restaurants_Main.csv")
#  creating csv for information
driver.close()

In [10]:
home_delivery = []
takeaway_available = []
vegetarian_only = []
indoor_seating = []
location = []
outlets = []

In [11]:
#urls=['https://www.zomato.com/hyderabad/wow-momo-dilsukhnagar']
print(home_delivery)
print(takeaway_available)
print(vegetarian_only)
print(indoor_seating)
for u in urls: 
    print(u)
    driver = webdriver.Chrome(options = chrome_options)
    driver.get(u)    
    rest_source = driver.page_source
    soup1 = BeautifulSoup(rest_source, "html.parser") 
    #get all the info below more info in restaurant url
    m = soup1.find("main")
    #print(m)
    
    #getting location of the restaurant
    a_tag = m.div.section.next_sibling.next_sibling.section.findAll("a")
    #print(a_tag)
    address = a_tag[-1].text #address of restaurant
    #print(address)
    location.append(address)
    
    #getting number of outlets
    outlet_info = m.div.section.next_sibling.next_sibling.next_sibling.next_sibling.findAll("article")
    outlet1 = outlet_info[2].findAll("a")
    no_outlets = re.findall('\d+',outlet1[-1].text)
    #print(no_outlets)
    if len(no_outlets)!=0:
        outlets.append(no_outlets[0])
    else:
        outlets.append(0)
    
    more_info=[]
    try:
        more_info = soup1.find(lambda tag: tag.name == "h3" and "More Info" in tag.text).next_element.next_element.findAll("p")
    except:
        pass   
    print("more info",more_info)
    more_info_list=[]
    if len(more_info)!=0:   #checking whether more info block is there or not 
        for info in more_info:
            more_info_list.append(info.text)
            #print(more_info_list)
            
        #checking presence of home delivery option
        if "Home Delivery" in more_info_list:
            home_delivery.append('yes')
        else:
            home_delivery.append('no')
        #checking presence of Takeaway Available option
        if "Takeaway Available" in more_info_list:
            takeaway_available.append('yes')
        else:
            takeaway_available.append('no')
        #checking presence of Indoor Seating option
        if "Indoor Seating" in more_info_list:
            indoor_seating.append('yes')
        else:
            indoor_seating.append('no')
        #checking presence of Vegetarian Only option
        if "Vegetarian Only" in more_info_list:
            vegetarian_only.append('yes')
        else:
            vegetarian_only.append('no')
    else:
        home_delivery.append('no')
        takeaway_available.append('no')
        indoor_seating.append('no')
        vegetarian_only.append('no')
        
print(len(home_delivery))
print(len(takeaway_available))
print(len(vegetarian_only))
print(len(indoor_seating))    
#print(location)
zomato_hyd = pd.read_csv('Hyderabad_restaurants_Main.csv')
zomato_hyd['location']=location
zomato_hyd['outlets']=outlets
zomato_hyd['home_delivery'] = home_delivery
zomato_hyd['takeaway_available'] = takeaway_available
zomato_hyd['vegetarian_only'] = vegetarian_only
zomato_hyd['indoor_seating'] = indoor_seating
zomato_hyd.to_csv("Hyderabad_restaurants_Main.csv")

driver.close()

[]
[]
[]
[]
https://www.zomato.com/hyderabad/fancy-ice-cream-1-charminar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Vegetarian Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Desserts and Bakes</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>]
https://www.zomato.com/hyderabad/peshawar-2-malakpet
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Family Friendly</p>]
https://www.zomato.com/hyderabad/nazeer-juice-center-charminar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cun

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>]
https://www.zomato.com/hyderabad/lucky-restaurant-saroor-nagar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Family Friendly</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>]
https://www.zomato.com/hyderabad/la-pinoz-pizza-dilsukhnagar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Family Friendly</p>]
https://www.zomato.com/hyderabad/scoops-ice-cream-2-charminar
more info [<p class="sc-1hez2tp-0

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Vegetarian Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Desserts and Bakes</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/tipsy-topsy-bakery-saroor-nagar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Breakfast</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Desserts and Bakes</p>]
https://www.zomato.com/hyderabad/mohammedia-shawarma-saroor-nagar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>]
https://www.zomato.com/hyderabad/hilton-restaurant-khairatabad
more info [<p class="

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Desserts and Bakes</p>]
https://www.zomato.com/hyderabad/mandi-king-arabian-restaurant-malakpet
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Desserts and Bakes</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Family Friendly</p>]
https://www.zomato.com/hyderabad/masqati-ice-cream-1-attapur
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Vegetarian Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Desserts and Bakes</p>]
https:/

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>]
https://www.zomato.com/hyderabad/pista-house-zafrani-tea-charminar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/spice-9-restaurant-malakpet
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Family Friendly</p>]
https://www.zomato.com/hyderabad/g-pulla-reddy-sweets-mehdipatnam
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="

...

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Vegetarian Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Desserts and Bakes</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/healthy-hunger-box-banjara-hills
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>]
https://www.zomato.com/hyderabad/house-of-curries-point-and-mess-dilsukhnagar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Breakfast</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/kiranmai-food-court-dilsukhnagar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Availa

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Breakfast</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/al-maskat-mandi-malakpet
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/al-kusum-mandi-malakpet
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/broast-n-roast-charminar
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/as-tandori-matka-chai-lakdikapul
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Breakfast</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/china-town-adikmet
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/shero-home-food-north-indian-malakpet
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Breakfast</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/chai-kings-dilsukhnagar
more info []
https://www.zomato.com/hyderabad/hotel-palace-1-mehdipatnam
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Breakfast</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" col

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Breakfast</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Vegetarian Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderabad/around-the-world-koti
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Vegetarian Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Outdoor Seating</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Desserts and Bakes</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>]
https://www.zomato.com/hyderabad/chefs-best-plates-mehdipatnam
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Delivery Only</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">No Seating Available</p>]
https://www.zomato.com/hyderab

more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Table reservation required</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Romantic Dining</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">4/5 Star</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Luxury Dining</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Valet Parking Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Outdoor Seating</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Family Friendly</p>]
https://www.zomato.com/hyderabad/citrus-cafe-lemon-tree-hotel-banjara-hills
more info [<p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Home Delivery</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Takeaway Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Valet Parking Available</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Indoor Seating</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">Buffet</p>, <p class="sc-1hez2tp-0 cunMUz" color="#4F4F4F">4/5 Star</p>, <p class="sc-1he

In [12]:
zomato_hyd.head()

,Unnamed: 0,links,names,ratings,price for one,cuisine,no of cisine,delivery time,location,outlets,home_delivery,takeaway_available,vegetarian_only,indoor_seating
0,0,https://www.zomato.com/hyderabad/fancy-ice-cre...,Fancy Ice Cream,4.1,₹100 for one,"Desserts, Ice Cream",2,28 min,"Charminar, Hyderabad",3,yes,yes,yes,yes
1,1,https://www.zomato.com/hyderabad/peshawar-2-ma...,Peshawar,4.1,₹100 for one,"North Indian, Seafood, Kebab, Chinese",4,37 min,"Malakpet, Hyderabad",2,yes,yes,no,yes
2,2,https://www.zomato.com/hyderabad/nazeer-juice-...,Nazeer Juice Center,4.2,₹100 for one,"Beverages, Juices, Healthy Food",3,27 min,"Charminar, Hyderabad",0,yes,yes,yes,yes
3,3,https://www.zomato.com/hyderabad/pista-house-2...,Pista House,4.1,₹100 for one,"North Indian, Chinese, Biryani, Kebab, Seafood...",8,35 min,"Charminar Area, Charminar, Hyderabad",23,yes,yes,no,yes
4,4,https://www.zomato.com/hyderabad/hotel-nayaab-...,Hotel Nayaab,4.0,₹100 for one,"North Indian, Mughlai, Chinese, Beverages",4,29 min,"Charminar, Hyderabad",0,yes,yes,no,yes


In [17]:
import joblib
joblib.dump(zomato_hyd,r"./zomato_webscrape.pkl")
zomato_pkl = joblib.load(r"./zomato_webscrape.pkl")

In [18]:
zomato_hyd = zomato_pkl.copy()

In [19]:
zomato_hyd.head()

,Unnamed: 0,links,names,ratings,price for one,cuisine,no of cisine,delivery time,location,outlets,home_delivery,takeaway_available,vegetarian_only,indoor_seating
0,0,https://www.zomato.com/hyderabad/fancy-ice-cre...,Fancy Ice Cream,4.1,₹100 for one,"Desserts, Ice Cream",2,28 min,"Charminar, Hyderabad",3,yes,yes,yes,yes
1,1,https://www.zomato.com/hyderabad/peshawar-2-ma...,Peshawar,4.1,₹100 for one,"North Indian, Seafood, Kebab, Chinese",4,37 min,"Malakpet, Hyderabad",2,yes,yes,no,yes
2,2,https://www.zomato.com/hyderabad/nazeer-juice-...,Nazeer Juice Center,4.2,₹100 for one,"Beverages, Juices, Healthy Food",3,27 min,"Charminar, Hyderabad",0,yes,yes,yes,yes
3,3,https://www.zomato.com/hyderabad/pista-house-2...,Pista House,4.1,₹100 for one,"North Indian, Chinese, Biryani, Kebab, Seafood...",8,35 min,"Charminar Area, Charminar, Hyderabad",23,yes,yes,no,yes
4,4,https://www.zomato.com/hyderabad/hotel-nayaab-...,Hotel Nayaab,4.0,₹100 for one,"North Indian, Mughlai, Chinese, Beverages",4,29 min,"Charminar, Hyderabad",0,yes,yes,no,yes
